NER

In [1]:
import cv2 

import pytesseract
import re

import difflib

from dateutil import parser
from spellchecker import SpellChecker

import datefinder
from datetime import datetime

import requests
import numpy as np
from io import BytesIO
from PIL import Image

In [2]:
def spellChecker(document, businessName, businessOwner, business_address):
    
    custom_list = []
    businessOwner = businessOwner.split()
    business_address = business_address.split()
    businessName = businessName.split()
    for name in businessOwner:
        custom_list.append(name)
    
    for add in business_address:
        custom_list.append(add)
        
    for bus in businessName:
        custom_list.append(bus)
        
    matched_lists = []
    def correct_name_spelling(sentence, name_list):
        # Convert sentence to lowercase
        sentence = sentence.lower()
        # Remove punctuation marks
        sentence = re.sub(r'[^\w\s]','',sentence)
        words = sentence.split()
        for i, word in enumerate(words):
            word = word.lower()
            name_list = [name.lower() for name in name_list]
            match = difflib.get_close_matches(word, name_list, n=1, cutoff=0.7)
            if not match:
                # check for the variations of the word
                variations = [word+'s', word+'es', word[:-1], word+'ed']
                for var in variations:
                    match = difflib.get_close_matches(var, name_list, n=1, cutoff=0.7)
                    if match:
                        break
            if match:
                matched_list = []
               # print(words[i].upper(), match[0])
                matched_list.append(words[i].upper())
                matched_list.append(match[0])
                #return the matched images
                words[i] = match[0]
                matched_lists.append(matched_list)
        corrected_sentence = " ".join(words)
        return corrected_sentence
    
    namedChecked = correct_name_spelling(document, custom_list)
    print(matched_lists)
    
    # Create a SpellChecker object
    spell = SpellChecker()

    # Your string with misspelled words
    misspelled_string = namedChecked

    # Find all misspelled words in the string
    misspelled_words = spell.unknown(misspelled_string.split())

    corrected_string = misspelled_string
    for word in misspelled_words:
        corrected_string = corrected_string.replace(word, spell.correction(word))

    return (corrected_string)
    

In [3]:
def ocrFunction(networkURL, bussinessName, businessOwner, business_address):
    
    cert = cv2.imread(networkURL, 0)
    ''' for network link
    ert = Image.open(BytesIO(requests.get(networkURL).content))
    ert = np.array(cert)
    '''
    thresh, binarized_cert =cv2.threshold(cert, 120, 200, cv2.THRESH_BINARY) #numbers are paramters that can be changed 
    
    
    pytesseract_result = pytesseract.image_to_string(binarized_cert) 
    pytesseract_result = pytesseract_result.replace('\n', ' ')
    pytesseract_result = " ".join(pytesseract_result.split())
    pytesseract_result = pytesseract_result.replace('_', '')
    
    text = spellChecker(pytesseract_result, bussinessName, businessOwner, business_address)
    
    matches = datefinder.find_dates(text)
    
    # Create an empty set
    unique_dates = set() #store date for date verification

    # Iterate over the matches
    for match in matches:
        date = match.date()
        # Check if the date is within the desired range
        if date.year >= 2000 and date.year <= 2030:
            # Add the date to the set, since sets don't allow duplicate elements, it will only store unique date strings.
            unique_dates.add(date.strftime("%Y-%m-%d"))

    # Print each unique dates
    '''
    for date in unique_dates:
        print(date)    
    '''
    print(unique_dates)

In [4]:
#network_url = 'https://firebasestorage.googleapis.com/v0/b/artemis-f2a4c.appspot.com/o/companyLogo%2FZinnovare%2FArchitecture%20of%208086.png?alt=media&token=56e6033a-eea7-4f70-839c-6abb4db20a28'
network_url = 'permit.png'
business_owner_fullname = "Marissa Berdaje Ugay"
business_name = "Bossilog Tapsilogan"
business_address = "Filinvest Road1 BHQC"

ocrFunction(network_url, business_name, business_owner_fullname, business_address)

[['MARISSA', 'marissa'], ['BERDAIE', 'berdaje'], ['UGAY', 'ugay'], ['FILINVEST', 'filinvest'], ['ROAD', 'road1'], ['BHOC', 'bhqc'], ['BOSSILOGTAPSILOGAN', 'tapsilogan']]
{'2020-12-31', '2021-01-20'}
